<a href="https://colab.research.google.com/github/Material-Educativo/Tecnicas-heuristicas/blob/main/Clustering_IRIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Cargar dataset IRIS
iris = datasets.load_iris()
X_iris = iris.data  # Características (150 x 4)
y_iris = iris.target  # Etiquetas verdaderas (solo para evaluación)
nombres_caracteristicas = iris.feature_names
nombres_especies = iris.target_names

print("Dataset IRIS cargado:")
print(f"  Número de muestras: {X_iris.shape[0]}")
print(f"  Número de características: {X_iris.shape[1]}")
print(f"  Características: {nombres_caracteristicas}")
print(f"  Especies: {nombres_especies}")

In [ ]:
# Normalizar datos (media 0, varianza 1)
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_iris)

print("\nEstadísticas después de normalización:")
print(f"  Media: {np.mean(X_norm, axis=0)}")
print(f"  Desviación estándar: {np.std(X_norm, axis=0)}")

#Visualización de IRIS

In [ ]:
# Aplicar PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_norm)

print(f"\nVarianza explicada por componentes principales:")
print(f"  PC1: {pca.explained_variance_ratio_[0]:.1%}")
print(f"  PC2: {pca.explained_variance_ratio_[1]:.1%}")
print(f"  Total: {sum(pca.explained_variance_ratio_):.1%}")


In [ ]:
# Visualizar datos en espacio de componentes principales
fig, ax = plt.subplots(figsize=(10, 7))

# Graficar cada especie con diferente color
colores = ['red', 'green', 'blue']
marcadores = ['o', 's', '^']

for i, especie in enumerate(nombres_especies):
    mask = y_iris == i
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
              c=colores[i], marker=marcadores[i],
              label=especie, alpha=0.7, s=100,
              edgecolors='black', linewidth=0.5)

ax.set_xlabel(f'Primera Componente Principal ({pca.explained_variance_ratio_[0]:.1%} varianza)', fontsize=12)
ax.set_ylabel(f'Segunda Componente Principal ({pca.explained_variance_ratio_[1]:.1%} varianza)', fontsize=12)
ax.set_title('Dataset IRIS - Proyección en Componentes Principales', fontsize=14, weight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

#Generación de soluciones aleatorias

In [ ]:
def calcular_costo(X, centroides, asignaciones):
    # Suma de distancias euclidianas de cada punto a su centroide asignado
    distancias = np.linalg.norm(X - centroides[asignaciones], axis=1)
    return distancias.sum()

def asignar_centroides(X, centroides):
    # Calcula la distancia de cada punto a cada centroide
    distancias = np.linalg.norm(X[:, np.newaxis, :] - centroides[np.newaxis, :, :], axis=2)
    # Asigna cada punto al centroide más cercano
    return np.argmin(distancias, axis=1)

In [ ]:
k = 3  # Número de clusters
n_inicializaciones = 100  # Número de soluciones iniciales aleatorias

# Rango de valores para centroides basado en el mínimo y máximo de cada característica
mins = X_norm.min(axis=0)
maxs = X_norm.max(axis=0)

costos = []  # Lista para almacenar el costo de cada inicialización

mejor_costo = np.inf  # Inicializar el mejor costo como infinito
mejores_centroides = None  # Para almacenar los centroides de la mejor solución
mejor_asignacion = None  # Para almacenar la asignación de puntos de la mejor solución

for _ in range(n_inicializaciones):
    # Generar centroides aleatorios dentro del rango de datos
    centroides_aleatorios = np.random.uniform(low=mins, high=maxs, size=(k, X_norm.shape[1]))

    # Asignar cada punto al centroide más cercano
    asignaciones = asignar_centroides(X_norm, centroides_aleatorios)

    # Calcular el costo total (suma de distancias)
    costo = calcular_costo(X_norm, centroides_aleatorios, asignaciones)

    costos.append(costo)  # Guardar el costo para análisis

    # Actualizar la mejor solución si el costo actual es menor
    if costo < mejor_costo:
        mejor_costo = costo
        mejores_centroides = centroides_aleatorios.copy()
        mejor_asignacion = asignaciones.copy()

costos = np.array(costos)  # Convertir la lista a un array de NumPy para análisis estadístico

print(f"Costo promedio: {np.mean(costos):.2f}")
print(f"Costo mínimo: {mejor_costo:.2f}")
print(f"Costo máximo: {np.max(costos):.2f}")

## Analicemos variabilidad con histogramas y diagramas de cajas.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Histograma de costos totales
ax1.hist(costos, bins=30, color='skyblue',
         edgecolor='black', alpha=0.8)
ax1.axvline(np.min(costos), color='green', linestyle='--',
            label=f'Mejor: {np.max(costos):.0f}')
ax1.axvline(np.mean(costos), color='red', linestyle='--',
            label=f'Promedio: {np.mean(costos):.0f}')
ax1.set_xlabel('Costo total')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de costos en calendarios aleatorios')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Diagrama de caja (boxplot)
ax2.boxplot(costos, vert=True)
ax2.set_ylabel('Costo total')
ax2.set_title('Resumen estadístico')
ax2.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Visualización del clustering usando la mejor asignación encontrada

In [ ]:
# Supongamos que 'mejor_asignacion' es un array con los índices de cluster para cada punto
fig, ax = plt.subplots(figsize=(10, 7))

colores = ['red', 'green', 'blue']
marcadores = ['o', 's', '^']

# Graficar cada cluster por separado
for i in range(3):
    mask = (mejor_asignacion == i)
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
               c=colores[i], marker=marcadores[i],
               label=f'Cluster {i+1}', alpha=0.7, s=100,
               edgecolors='black', linewidth=0.5)

# Configuración de etiquetas y título
ax.set_xlabel(f'Primera Componente Principal ({pca.explained_variance_ratio_[0]:.1%} varianza)', fontsize=12)
ax.set_ylabel(f'Segunda Componente Principal ({pca.explained_variance_ratio_[1]:.1%} varianza)', fontsize=12)
ax.set_title('Clustering IRIS - Mejores asignaciones en espacio PCA', fontsize=14, weight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
